In [ ]:
import tensorflow as tf

#Check if a GPU is available as accelerator
if tf.config.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True)
    print('Using GPU')
else:
    print('Using CPU')

In [ ]:
import skimage
import tifffile
from skimage import io
from matplotlib import pyplot as plt
import numpy as np
from scipy import ndimage
import timeit

In [ ]:
#Upload segmentations (axial, coronal, and sagittal views) obtained from the test MRI scan
axial = io.imread('axial_reconstructed_image.tif')
coronal = io.imread('coronal_reconstructed_image.tif')
sagittal = io.imread('sagittal_reconstructed_image.tif')

mask = io.imread('data/Scansione1/320maskStack.tif')

In [ ]:
# Perform morphological operations to generate different views from axial slices
def get_views_from_axial(image):
    image = image.astype(np.uint8)
    sagittal = np.rot90(image, k=2, axes=(2, 0))
    sagittal = np.fliplr(np.rot90(np.transpose(sagittal, [0, 2, 1])))

    coronal = np.rot90(image, axes=(0, 1))

    return sagittal, coronal

# Perform morphological operations to generate different views from sagittal slices
def get_views_from_sagittal(image):
    axial = np.transpose(image, (1, 2, 0))

    coronal = np.rot90(axial, axes=(0, 1))

    return axial, coronal

In [ ]:
axial_2_sagittal, axial_2_coronal = get_views_from_axial(axial)

sagittal_2_axial, sagittal_2_coronal = get_views_from_sagittal(sagittal)

In [ ]:
#Combine the three segmentation outputs using logical OR
binary_output = np.logical_or(axial_2_coronal, sagittal_2_coronal)
binary_output = np.logical_or(binary_output, coronal)

In [ ]:
import cupy as cp
from cupyx.scipy.ndimage import binary_dilation, binary_erosion, binary_closing, label
from skimage import measure
from skimage.morphology import ball
from skimage.morphology import binary_closing as binary_closing_cpu

#Extract connected components (using GPU as accelerator)
def connected_components_gpu(image_stack, iterations, closing):
    shift = 15
    image_stack = np.roll(image_stack,shift,axis=1)
    image_stack = cp.asarray(image_stack)
    selem = cp.asarray(ball(9))

    for i in range(iterations):
        # Trova le componenti connesse nell'immagine usando cupyx.scipy.ndimage.label
        labeled_stack, num_features = label(image_stack)

        # Calcola le proprietà delle componenti connesse usando regionprops di scikit-image
        props = measure.regionprops(cp.asnumpy(labeled_stack))

        # Trova l'indice della componente connessa più grande
        largest_label = cp.argmax(cp.array([prop.area for prop in props])) + 1

        # Estrai la componente connessa più grande
        largest_component = (labeled_stack == largest_label)
        
        if closing:
            closed_largest_component = binary_closing(largest_component, structure=selem)
        else:
            closed_largest_component = largest_component
        
        image_stack = cp.logical_xor(image_stack, largest_component)

        if i == 0:
            result = closed_largest_component
        else:
            result = cp.logical_or(result, closed_largest_component)
        
    result = cp.asnumpy(result)
    result = np.roll(result,-shift,axis=1)
    
    return cp.asnumpy(result)

#Extract connected components
def connected_components(image_stack, iterations, closing):

    for i in range(iterations):
        
        selem = ball(9)
        
        # Trova le componenti connesse nell'immagine
        labeled_stack = measure.label(image_stack)

        # Calcola le proprietà delle componenti connesse
        props = measure.regionprops(labeled_stack)

        # Trova l'indice della componente connessa più grande
        largest_label = np.argmax([prop.area for prop in props]) + 1

        # Estrai la componente connessa più grande
        largest_component = (labeled_stack == largest_label)
        
        if closing==True:
            closed_largest_component = binary_closing_cpu(largest_component, selem)
        else:
            closed_largest_component = largest_component
            
        image_stack = np.logical_xor(image_stack,largest_component)

        if i == 0:
            result = closed_largest_component
        else:
            result = np.logical_or(result,closed_largest_component)
    
    return result

In [ ]:
t0 = timeit.default_timer()
final_segmentation = connected_components_gpu(binary_output,iterations = 3, closing = True)
t1 = timeit.default_timer()

print(f"Time: {t1 - t0} s")

In [ ]:
from glob import glob
from tqdm import tqdm
import tifffile

#If already got the result
#pred_mask = io.imread('/kaggle/input/final_segmentation.tif')

pred_mask = final_segmentation

true_sagittal, true_coronal = get_views_from_axial(mask)

In [ ]:
# Convert boolean masks to integer arrays for further processing
def bool_to_int(stack):
    lista_slice = []
    
    for slice in stack:
        slice_convertita = np.where(slice, 1, 0)
        lista_slice.append(slice_convertita.tolist())
    
    array_stack = np.array(lista_slice)
    return array_stack

In [ ]:
pred_mask = bool_to_int(pred_mask)
true_mask = bool_to_int(true_coronal)

print(pred_mask.shape)

In [ ]:
#Visualize differences between predicted mask and ground truth

difference = pred_mask.astype(int) - true_mask.astype(int)

color_image = np.zeros((*difference.shape, 3), dtype=np.uint8)

#Green = False Positives
color_image[difference == 1] = [0, 255, 0]

#Red = False Negatives
color_image[difference == -1] = [255, 0, 0]

plt.imshow(color_image[250,:,:])

In [ ]:
import glob
import os

import numpy as np
import pymia.evaluation.metric as metric
import pymia.evaluation.evaluator as eval_
import pymia.evaluation.writer as writer
import SimpleITK as sitk

In [ ]:
#Define evaluation metrics using Pymia
metrics = [metric.DiceCoefficient(metric='DSC'), 
           metric.Sensitivity(metric='Sens'),
           metric.Specificity(metric='Spec'), 
           metric.VolumeSimilarity(metric='VS'), 
           metric.AverageDistance(metric='AVD'), 
           metric.SurfaceOverlap(metric='SO'),
           metric.Accuracy(metric='AVG'),
           metric.FMeasure(metric='F1'),
           metric.JaccardCoefficient(metric='IoU'),
           metric.Precision(metric='Prec')] 

labels = {1: 'BONE'}
evaluator = eval_.SegmentationEvaluator(metrics, labels)

In [ ]:
prediction = sitk.GetImageFromArray(pred_mask)
ground_truth = sitk.GetImageFromArray(true_mask)

print(f'Evaluating...')
evaluator.evaluate(prediction,ground_truth,1)

In [ ]:
result_file = '/kaggle/working/results.csv'
writer.CSVWriter(result_file).write(evaluator.results)
evaluator.clear()

In [ ]:
import pandas as pd
pd.read_csv(result_file,sep=';')

In [ ]:
#Extract the three largest connected components (individual bones)
def get_bones(image_stack):
    for i in range(3):
        labeled_stack = measure.label(image_stack)

        props = measure.regionprops(labeled_stack)

        largest_label = np.argmax([prop.area for prop in props]) + 1

        largest_component = (labeled_stack == largest_label)

        image_stack = np.logical_xor(image_stack,largest_component)
        
        if i==0:
            CC1 = bool_to_int(largest_component)
        elif i==1:
            CC2 = bool_to_int(largest_component)
        elif i==2:
            CC3 = bool_to_int(largest_component)

    return CC1,CC2,CC3

In [ ]:
pred_CC1,pred_CC2,pred_CC3 = get_bones(pred_mask)
true_CC1,true_CC2,true_CC3 = get_bones(true_mask)

In [ ]:
plt.subplot(1,2,1)
plt.imshow(pred_CC3[340,:,:],cmap='gray')

plt.subplot(1,2,2)
plt.imshow(true_CC3[340,:,:],cmap='gray')

In [ ]:
# Run evaluation on a single extracted bone
prediction = sitk.GetImageFromArray(pred_CC3) 
ground_truth = sitk.GetImageFromArray(true_CC3)

print(f'Evaluating...')
evaluator.evaluate(prediction,ground_truth,1)